<a href="https://colab.research.google.com/github/WideSu/CS608RecommendationSystem/blob/main/SPOP_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/SerenRec/
data_dir = './data/'
model_dir = './seren/model/'
import pandas as pd
train_data = pd.read_csv(data_dir+'train_data.csv')
test_data = pd.read_csv(data_dir+'test_data.csv')
# sys.path.append("./seren/data/")
from seren.data.Preprocess import *
from seren.data.dataset import * 

Mounted at /content/drive
/content/drive/MyDrive/Github/SerenRec


In [2]:
preprocessor = Preprocess()
train_seq = preprocessor.to_sequence(df = train_data)
test_seq = preprocessor.to_sequence(df = test_data)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [3]:
train_seq

,ss_id,sequence,next
0,0,"[3, 0, 0, 0, 0]",4
1,1,"[3, 4, 0, 0, 0]",4
2,2,"[3, 4, 4, 0, 0]",5
3,3,"[3, 4, 4, 5, 0]",6
4,4,"[4, 0, 0, 0, 0]",4
...,...,...,...
460947,460947,"[6715, 10063, 11215, 11417, 0]",11883
460948,460948,"[8455, 0, 0, 0, 0]",8068
460949,460949,"[8455, 8068, 0, 0, 0]",4309
460950,460950,"[8455, 8068, 4309, 0, 0]",7907


In [4]:
train_dataset = SessionDataset(
    seq_list = train_seq['sequence'], 
    next_list= train_seq['next'], sample_cnt=0,item_id_map = {11943: 0, 1: 1})

In [5]:
test_dataset = SessionDataset(
    seq_list = test_seq['sequence'], 
    next_list= test_seq['next'], 
    sample_cnt=0,
    item_id_map = {11943: 0, 1: 1})

In [6]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_data)
test_dataloader = DataLoader(test_data)

In [7]:
item_num = train_data['item_id'].nunique()
session_num = len(train_seq)

In [8]:
item_seq, next_item, _ = next(train_dataset.get_loader())

In [9]:
item_seq

[3, 0, 0, 0, 0]

In [14]:
import sys, os
import torch
from torch.utils.data import DataLoader
# sys.path.append("./seren/model/")
from seren.model.spop import *
config = {
    'max_len' : 5,
    'item_num' : item_num,
    'device' : "cuda" if torch.cuda.is_available() else "cpu"
}
spop = SessionPop(config)
train_dataloader = DataLoader(train_dataset, batch_size=512)
spop.fit(train_dataloader)

100%|██████████| 901/901 [00:20<00:00, 44.44it/s]


In [15]:
test_dataloader = DataLoader(test_dataset, batch_size=512)

In [17]:
res_ids, res_scs = spop.rank(test_dataloader)

100%|██████████| 40/40 [00:37<00:00,  1.06it/s]
